## Constraints

In [ ]:
// constr Load Author
CREATE CONSTRAINT author_name_unique FOR (author:Author) REQUIRE author.name IS UNIQUE

In [ ]:
// constr Load Conference
CREATE CONSTRAINT conference_name_unique ON (conf:Conference) ASSERT conf.name IS UNIQUE

In [ ]:
// constr Load Institution
CREATE CONSTRAINT institution_name_unique FOR (inst:Institution) REQUIRE inst.institution IS UNIQUE

In [ ]:
//constr Load Journal
CREATE CONSTRAINT journal_name_unique FOR (j:Journal) REQUIRE j.name IS UNIQUE

In [ ]:
//constr Load Publication
CREATE CONSTRAINT publication_title_unique FOR (pc:Publication) REQUIRE pc.doi IS UNIQUE

In [ ]:
//constr Load Publisher
CREATE CONSTRAINT publisher_name_unique ON (ps:Publisher) ASSERT ps.name IS UNIQUE

## Entità 

In [ ]:
// load auth
LOAD CSV WITH HEADERS FROM 'file:///author.csv' AS row
FIELDTERMINATOR "|" WITH row
WITH row WHERE row.name IS NOT NULL
MERGE (a:Author {name: row.name, orcid:row.orcid});

In [ ]:
// load conf
LOAD CSV WITH HEADERS FROM 'file:///conferences.csv' AS row
FIELDTERMINATOR ";"
WITH row WHERE row.name IS NOT NULL
MERGE (con:Conference {name: row.name, year: toIntegerOrNull(row.year)});

In [ ]:
// load inst
LOAD CSV WITH HEADERS FROM 'file:///institution.csv' AS row
FIELDTERMINATOR '|'
WITH row WHERE row.institution IS NOT NULL
CREATE (inst:Institution {  wr: toIntegerOrNull(row.world_rank),
                            name_uni: row.institution,
                            country: row.country,
                            nr: toIntegerOrNull(row.national_rank),
                            qe: toIntegerOrNull(row.quality_of_education), 
                            score: toIntegerOrNull(row.score)
                            });

In [ ]:
// load jour
LOAD CSV WITH HEADERS FROM 'file:///journals.csv' AS row
WITH row WHERE row.name IS NOT NULL
MERGE (j:Journal {name: row.name});

In [ ]:
// load pub
LOAD CSV WITH HEADERS FROM 'file:///publications.csv' AS row
FIELDTERMINATOR '|'
CREATE (p:Publication {
                        doi: row.id,
                        title: row.title,
						year: toIntegerOrNull(coalesce(row.year, '')),
					    pages: toIntegerOrNull(coalesce(row.pages, '')), 
                        isbn: coalesce(row.isbn, ''), 
                        publisher: coalesce(row.publisher, ''), 
                        doc_type: row.doc_type});

                

In [ ]:
// load publisher
LOAD CSV WITH HEADERS FROM 'file:///publisher.csv' AS row
WITH row WHERE row.name IS NOT NULL
MERGE (ps:Publisher {name: row.name});

## Load Relationship

In [ ]:
//Load Relation Citation 
LOAD CSV WITH HEADERS FROM "file:///citationFAKE.csv" AS row FIELDTERMINATOR "|" 
MATCH (a:Publication{doi:row.document}),(b:Publication{doi:row.cite})
CREATE (a)-[:CITE]->(b)

In [ ]:
//Load Relation Hold   
LOAD CSV WITH HEADERS FROM "file:///conferences_relationship.csv" AS row FIELDTERMINATOR "|"
MATCH (a:Publication{doi:row.id})
MATCH (b:Conference{name:row.name})
CREATE (a)-[:HOLD]->(b)

In [ ]:
//Load Relation Journal
LOAD CSV WITH HEADERS FROM "file:///journals_relationship.csv" AS row FIELDTERMINATOR "|"
MATCH (a:Publication{doi:row.id}),(b:Journal{name:row.name})
CREATE (a)-[:CONTAINED]->(b)

In [ ]:
//Load relation editors 
LOAD CSV WITH HEADERS FROM "file:///editors_relationship.csv" AS row
FIELDTERMINATOR "|"
MATCH (p:Publication {doi: row.doi})
MERGE (e:Editor {name: row.editor})
MERGE (e)-[:EDITED]->(p)

In [ ]:
//Load relation keywords 
LOAD CSV WITH HEADERS FROM "file:///keywords.csv" AS row
FIELDTERMINATOR ";"
MATCH (p:Publication {doi: row.pubID})
MERGE (k:Keyword {name: row.keyword})
MERGE (p)-[:KEYWORD]->(k)

In [ ]:
//Load Relation Publisher
LOAD CSV WITH HEADERS FROM "file:///publisher_relationship.csv" AS row FIELDTERMINATOR "|"
MATCH (a:Publication{doi:row.id})
MATCH (b:Publisher{name:row.name})
CREATE (a)-[:PUBLISH]->(b)

In [ ]:
//load relation work
LOAD CSV WITH HEADERS FROM "file:///work_rel.csv" AS row
FIELDTERMINATOR "|"
MATCH (a:Author {name: row.name})
MATCH (inst:Institution {name_uni: row.university})
CREATE (a)-[:WORK]->(inst)

In [ ]:
//Load Relation Write
load csv with headers from "file:///relationship.csv" AS row FIELDTERMINATOR "|" 
match(a:Author{name:row.author_name})
match(p:Publication{doi:row.id})
create (a)-[:WROTE{order:row.author_order}]->(p)